In [1]:
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from scipy.stats import rankdata 
import ast
import os


fig_dir = './figs/'
os.makedirs(fig_dir, exist_ok=True)

df_all = pd.read_csv('paper-figure_celllist_all.csv')
print(f'Number of samples: {df_all.shape[0]}')

custom_stack_order = ['Squ.epi', 'Squ.meta', 'Navicular', 'Para.Squ', 'Para.Clust', 'Glan', 'Leu','Debris', 'LSIL', 'HSIL', 'Adenocarcinoma']
legend_stack_order = ['Leu', 'Squ.epi', 'Navicular', 'Para.Squ', 'Squ.meta', 'Glan', 'Para.Clust', 'LSIL', 'HSIL', 'Adenocarcinoma', 'Debris']
legend_labels= ['Leukocyte', 'Superficial/intermediate cell', 'Navicular cell', 'Parabasal cell', 'Squamous metaplasia', 'Glandular cell', 'Miscellaneous cell cluster', 'LSIL', 'HSIL', 'Adenocarcinoma', 'Irrelevant object']
CLASS_ORDER = ["NILM", "ASC-US", "LSIL", "ASC-H", "HSIL", "SCC", "AGC"]


probability_threshold = 0.8
for param in custom_stack_order:
    df_all[param] = df_all[f'class_count-{probability_threshold}'].apply(ast.literal_eval).apply(lambda x: x.get(param, 0))

Number of samples: 1124


In [2]:
def summarize_counts(df, metric_col):
    def _agg(g):
        v = g[metric_col].dropna().astype(float).values
        if v.size == 0:
            return pd.Series({"n":0,"median":np.nan,"q1":np.nan,"q3":np.nan,
                              "mean":np.nan,"sd":np.nan,"min":np.nan,"max":np.nan})
        return pd.Series({
            "n": v.size,
            "median": np.median(v),
            "q1": np.percentile(v, 25),
            "q3": np.percentile(v, 75),
            "mean": v.mean(),
            "sd": v.std(ddof=1) if v.size > 1 else 0.0,
            "min": v.min(),
            "max": v.max()
        })
    return (df.groupby(["Facility","Class"], dropna=False)
              .apply(_agg).reset_index())

def bh_correction_vector(p):
    """
    Return Benjamini-Hochberg adjusted q-values for a 1D array of p-values (keeping NaNs).
    """
    p = np.asarray(p, dtype=float)
    q = np.full_like(p, np.nan, dtype=float)

    mask = ~np.isnan(p)
    m = mask.sum()
    if m == 0:
        return q

    idx = np.flatnonzero(mask)    
    p_masked = p[idx]
    order = np.argsort(p_masked, kind="mergesort")
    ranked = p_masked[order] * (m / np.arange(1, m + 1))
    ranked[::-1] = np.minimum.accumulate(ranked[::-1]) 
    q_masked = np.clip(ranked, 0, 1)

    q[idx[order]] = q_masked
    return q


# --- Calculate effect size ---

def _effects_vs_nilm(x: np.ndarray, y: np.ndarray):
    """
    x: NILM (reference), y: comparator. Positive values mean y > x.
    """
    n1, n2 = x.size, y.size
    N = n1 + n2
    if n1 == 0 or n2 == 0:
        return dict(A=np.nan, delta=np.nan, r_rb=np.nan,
                    d=np.nan, g=np.nan, r_z=np.nan)

    # --- Calcurate U_ref from rank sum (ties -> average ranks) ---
    pooled = np.concatenate([x, y])
    ranks  = rankdata(pooled, method="average")
    R_ref  = ranks[:n1].sum()
    U_ref  = R_ref - n1 * (n1 + 1) / 2.0

    # Vargha–Delaney A for comparator (y): A_y = 1 - U_ref/(n1*n2)
    A_y   = 1.0 - (U_ref / (n1 * n2))
    delta = 2.0 * A_y - 1.0                 # Cliff's δ (= rank-biserial r)

    # --- Cohen's d / Hedges' g（そのまま） ---
    N = n1 + n2
    mean_x, mean_y = np.mean(x), np.mean(y)
    sd_x = np.std(x, ddof=1) if n1 > 1 else np.nan
    sd_y = np.std(y, ddof=1) if n2 > 1 else np.nan
    if n1 > 1 and n2 > 1:
        sp = np.sqrt(((n1 - 1) * sd_x**2 + (n2 - 1) * sd_y**2) / (N - 2))
        d = (mean_y - mean_x) / sp if sp > 0 else np.nan
        J = 1.0 - 3.0 / (4.0 * N - 9.0) if N > 2 else 1.0
        g = J * d
    else:
        d = np.nan; g = np.nan

    # --- r from z ---
    # var(U) = (n1*n2/12) * [(N+1) - sum(t_k^3 - t_k)/(N(N-1))]
    _, cnts = np.unique(pooled, return_counts=True)
    T = np.sum(cnts**3 - cnts) if N > 1 else 0.0
    sigma2 = (n1 * n2 / 12.0) * ((N + 1.0) - T / (N * (N - 1.0))) if N > 1 else np.nan
    mu = n1 * n2 / 2.0
    if sigma2 and sigma2 > 0:
        z = (U_ref - mu) / np.sqrt(sigma2)
        r_z = (-z) / np.sqrt(N) 
    else:
        r_z = np.nan

    return dict(A=A_y, delta=delta, r_rb=delta, d=d, g=g, r_z=r_z)

def tests_vs_nilm_one_sided_with_bh(df, metric_col):
    rows = []
    for fac, g in df.groupby("Facility", dropna=False):
        if "NILM" not in g["Class"].unique():
            continue
        x = g.loc[g["Class"]=="NILM", metric_col].dropna().astype(float).values
        comps = [c for c in g["Class"].unique() if c != "NILM"]
        tmp, pvals = [], []
        for cls in comps:
            y = g.loc[g["Class"]==cls, metric_col].dropna().astype(float).values
            if x.size==0 or y.size==0:
                U = np.nan; p = np.nan
                eff = dict(A=np.nan, delta=np.nan, r_rb=np.nan, d=np.nan, g=np.nan, r_z=np.nan)
            else:
                # p ... one-sided (NILM group < comparison group)
                U, p = mannwhitneyu(x, y, alternative="less", method="auto")
                eff = _effects_vs_nilm(x, y)

            row = {"Facility": fac, "Class": cls,
                   f"U_{metric_col}": U, f"p_{metric_col}": p,
                   f"A_{metric_col}": eff["A"],
                   f"delta_{metric_col}": eff["delta"],
                   f"r_rb_{metric_col}": eff["r_rb"],
                   f"d_{metric_col}": eff["d"],
                   f"g_{metric_col}": eff["g"],
                   f"r_from_z_{metric_col}": eff["r_z"],
                   f"n_NILM_{metric_col}": x.size,
                   f"n_comp_{metric_col}": y.size}
            tmp.append(row)
            pvals.append(p)

        if tmp:
            qvals = bh_correction_vector(np.array(pvals, dtype=float))
            for r, qv in zip(tmp, qvals):
                r[f"q_BH_{metric_col}"] = qv
            rows.extend(tmp)
    return pd.DataFrame(rows)

def format_pq(v):
    if pd.isna(v): return ""
    if v < 1e-4:   return f"{v:.1e}"      # 1.2e-06
    if v < 0.1:    return f"{v:.3f}"      # 0.023
    return f"{v:.2f}"                     # 0.12

def _fmt_eff(v, k=3):
    if pd.isna(v): return ""
    return f"{v:.{k}f}"

def stars_from_q(q):
    if pd.isna(q): return ""
    return "***" if q < 1e-3 else ("**" if q < 1e-2 else ("*" if q < 5e-2 else ""))

def build_table(df_all, metric_col):
    summ  = summarize_counts(df_all, metric_col)
    tests = tests_vs_nilm_one_sided_with_bh(df_all, metric_col)
    out   = summ.merge(tests, on=["Facility","Class"], how="left")

    for col in [f"U_{metric_col}", f"p_{metric_col}", f"q_BH_{metric_col}",
                f"A_{metric_col}", f"delta_{metric_col}", f"r_rb_{metric_col}",
                f"d_{metric_col}", f"g_{metric_col}", f"r_from_z_{metric_col}",
                f"n_NILM_{metric_col}", f"n_comp_{metric_col}"]:
        if col in out.columns:
            out.loc[out["Class"]=="NILM", col] = np.nan

    present = [c for c in CLASS_ORDER if c in out["Class"].unique().tolist()]
    rest    = [c for c in out["Class"].unique().tolist() if c not in present]
    out["Class"] = pd.Categorical(out["Class"], categories=present + sorted(rest), ordered=True)
    out = out.sort_values(["Facility","Class"]).reset_index(drop=True)

    out["median [Q1-Q3]"] = out.apply(lambda r: f'{int(r["median"]):,} [{int(r["q1"]):,}-{int(r["q3"]):,}]' if pd.notna(r["median"]) else "", axis=1)
    out["mean ± sd"]       = out.apply(lambda r: f'{r["mean"]:.1f} ± {r["sd"]:.1f}' if pd.notna(r["mean"]) else "", axis=1)
    out["min-max"]         = out.apply(lambda r: f'{int(r["min"]):,}-{int(r["max"]):,}' if pd.notna(r["min"]) else "", axis=1)

    out["p value"]     = out[f"p_{metric_col}"].apply(format_pq)
    out["BH q value"]  = out[f"q_BH_{metric_col}"].apply(format_pq)
    out["significance"] = out[f"q_BH_{metric_col}"].apply(stars_from_q)

    out["A (comp > NILM)"]     = out[f"A_{metric_col}"].apply(lambda v: _fmt_eff(v, 3))
    out["Cliff's δ (comp>NILM)"] = out[f"delta_{metric_col}"].apply(lambda v: _fmt_eff(v, 3))
    out["r (rank-biserial)"]   = out[f"r_rb_{metric_col}"].apply(lambda v: _fmt_eff(v, 3))
    out["Cohen's d (y-NILM)"]  = out[f"d_{metric_col}"].apply(lambda v: _fmt_eff(v, 3))
    out["Hedges' g (y-NILM)"]  = out[f"g_{metric_col}"].apply(lambda v: _fmt_eff(v, 3))
    out["r (from Z)"]          = out[f"r_from_z_{metric_col}"].apply(lambda v: _fmt_eff(v, 3))

    out["n"] = pd.to_numeric(out["n"], errors="coerce").round().astype("Int64")

    cols = ["Facility","Class","n","median [Q1-Q3]","mean ± sd","min-max",
            "p value","BH q value","significance",
            "A (comp > NILM)","Cliff's δ (comp>NILM)","r (rank-biserial)",
            "Cohen's d (y-NILM)","Hedges' g (y-NILM)","r (from Z)"]
    return out[cols]

In [3]:
table_LSIL = build_table(df_all, "LSIL")
table_LSIL.to_csv(f'{fig_dir}STable1_LSIL-statistics.csv', index=False, encoding='utf-8-sig')
table_LSIL

/tmp/ipykernel_1414/1514120348.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_agg).reset_index())


,Facility,Class,n,median [Q1-Q3],mean ± sd,min-max,p value,BH q value,significance,A (comp > NILM),Cliff's δ (comp>NILM),r (rank-biserial),Cohen's d (y-NILM),Hedges' g (y-NILM),r (from Z)
0,1_CIH,NILM,169,1 [0-4],4.3 ± 10.6,0-72,,,,,,,,,
1,1_CIH,ASC-US,72,4 [2-9],7.2 ± 12.0,0-96,1.3e-07,2.1e-07,***,0.706,0.412,0.412,0.265,0.264,0.332
2,1_CIH,LSIL,42,34 [11-82],62.6 ± 85.1,2-437,1.2e-18,6.2e-18,***,0.928,0.855,0.855,1.498,1.493,0.601
3,1_CIH,ASC-H,9,4 [3-9],7.0 ± 7.7,1-26,0.004,0.004,**,0.758,0.517,0.517,0.255,0.253,0.202
4,1_CIH,HSIL,24,18 [10-86],53.5 ± 64.4,1-228,3.7e-11,9.1e-11,***,0.902,0.803,0.803,2.014,2.006,0.469
5,1_CIH,SCC,2,197 [117-276],197.0 ± 224.9,38-356,0.007,0.007,**,0.985,0.970,0.970,9.510,9.468,0.187
6,2_Tsukuba,NILM,55,2 [0-4],2.5 ± 2.7,0-10,,,,,,,,,
7,2_Tsukuba,ASC-US,31,8 [1-15],15.4 ± 31.7,0-177,0.000,0.000,***,0.738,0.476,0.476,0.676,0.670,0.399
8,2_Tsukuba,LSIL,54,50 [21-293],197.8 ± 299.6,"0-1,224",3.1e-16,1.6e-15,***,0.948,0.895,0.895,0.926,0.919,0.775
9,2_Tsukuba,ASC-H,17,3 [1-6],4.2 ± 4.6,0-16,0.099,0.099,,0.603,0.205,0.205,0.533,0.527,0.153


In [4]:
table_HSIL = build_table(df_all, "HSIL")
table_HSIL.to_csv(f'{fig_dir}STable1_HSIL-statistics.csv', index=False, encoding='utf-8-sig')
table_HSIL

/tmp/ipykernel_1414/1514120348.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_agg).reset_index())


,Facility,Class,n,median [Q1-Q3],mean ± sd,min-max,p value,BH q value,significance,A (comp > NILM),Cliff's δ (comp>NILM),r (rank-biserial),Cohen's d (y-NILM),Hedges' g (y-NILM),r (from Z)
0,1_CIH,NILM,169,0 [0-2],2.3 ± 6.0,0-65,,,,,,,,,
1,1_CIH,ASC-US,72,2 [0-4],3.7 ± 5.2,0-28,7.2e-05,0.000,***,0.646,0.292,0.292,0.247,0.247,0.245
2,1_CIH,LSIL,42,3 [1-7],4.7 ± 5.0,0-25,6.3e-07,1.6e-06,***,0.728,0.457,0.457,0.419,0.417,0.334
3,1_CIH,ASC-H,9,6 [2-14],13.9 ± 22.5,0-72,0.000,0.000,***,0.808,0.615,0.615,1.535,1.528,0.252
4,1_CIH,HSIL,24,29 [12-69],61.9 ± 104.3,0-517,1.5e-12,7.4e-12,***,0.914,0.828,0.828,1.629,1.622,0.502
5,1_CIH,SCC,2,340 [187-492],340.0 ± 431.3,35-645,0.004,0.004,**,0.997,0.994,0.994,10.018,9.973,0.201
6,2_Tsukuba,NILM,55,8 [3-24],28.5 ± 59.3,0-355,,,,,,,,,
7,2_Tsukuba,ASC-US,31,19 [7-36],38.7 ± 96.5,0-551,0.044,0.044,*,0.611,0.222,0.222,0.137,0.136,0.184
8,2_Tsukuba,LSIL,54,19 [5-51],41.1 ± 63.3,0-390,0.020,0.025,*,0.614,0.229,0.229,0.206,0.205,0.198
9,2_Tsukuba,ASC-H,17,47 [24-191],101.9 ± 103.9,5-319,3.3e-05,5.6e-05,***,0.822,0.644,0.644,1.020,1.009,0.471
